In [1]:
pip install chromedriver-autoinstaller


C:\Users\User\Web_Crawling\frizmworks>doskey defects4j=perl C:\Users\User\defects4j\framework\bin\defects4j $* 
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium


C:\Users\User\Web_Crawling\frizmworks>doskey defects4j=perl C:\Users\User\defects4j\framework\bin\defects4j $* 
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: certifi>=2021.10.8 in c:\users\user\anaconda3\lib\site-packages (from selenium) (2023.7.22)


In [3]:
import chromedriver_autoinstaller
from selenium import webdriver
import time

class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
#         options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.set_window_size(1600, 1100)

        return driver

In [4]:
driver = Driver.driver_init()

In [5]:
def find_product_paths(urls):
    product_paths = []
    
    ## 아우터/상의/하의 분류 루프
    for (url, pages_cnt) in urls:
        ## 각 옷 분류의 페이지 갯수 루프
        for page_num in range(pages_cnt):
            try:
                driver.get(url + "&page=" + str(page_num+1))
            except:
                continue

            products = driver.find_elements('xpath', '//ul[@class="prdList grid4"]/li')
            products_cnt = len(products)

            ## 각 product의 path를 product_paths 리스트에 저장
            for i in range(products_cnt):
                product = driver.find_elements('xpath', '//ul[@class="prdList grid4"]/li')[i]
                product_path = product.find_element('xpath', './div[@class="thumbnail"]/a').get_attribute('href')
                product_paths.append(product_path)
                
    return product_paths

In [6]:
def crawl_pages(product_paths):
    
    base_dict= {"baseCat":[], "baseName": [], "baseCode": [], "baseUrl": [], "colorName": [],
            "baseGender": [], "price": [], "basePrice": [], "colorFabric": [], "imageUrl": [],
            "size": [], "isAvailable": []
            }
    
    ## 드라이버 초기화
    driver = Driver.driver_init()
    ## 각 product에 대한 정보 추출
    for product_path in product_paths:
        driver.get(product_path)

        cat = 'top'
        if "category/24" in product_path or "category/52" in product_path:
            cat = 'top'
        elif "category/26" in product_path:
            cat = 'bottom'

        baseURL = product_path
        base_gender = 0
        text_info = driver.find_element('xpath', '//div[@class="infoArea"]')
        imageURL = driver.find_element('xpath','//img[@class="BigImage "]').get_attribute('src')

        base_name = text_info.find_element('xpath', './div[@class="headingArea"]/h2').text
        base_code = ""
        color_name = ""
        ## color가 명시되지 않은 옷은 제외(패키지 상품)
        if "_" in base_name:
            base_code = base_name.split("_")[0]
            ## base_code 앞에 협업 브랜드 텍스트 제거
            if "] " in base_code:
                base_code = base_code.split("] ")[1]

            ## color_name 뒤에 협업 브랜드 텍스트 제거
            color_name = base_name.split("_")[1]
            if "[" in color_name:
                color_name = color_name.split("[")[0]
        else:
            continue

        base_price = ""
        price = ""
        color_fabric = ""
        trs = text_info.find_elements("xpath",'./div[2]/table/tbody/tr') #body의 table 항목 갯수                       
        if len(trs) == 5:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = text_info.find_element("xpath",'.//span[@id="span_product_price_sale"]').text
            color_fabric = text_info.find_element("xpath",'./div[2]/table/tbody/tr[5]/td/span').text
        elif len(trs) == 4:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = base_price
            color_fabric = text_info.find_element("xpath",'./div[2]/table/tbody/tr[4]/td/span').text
        elif len(trs) == 3:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = base_price
            color_fabric = ""
        elif len(trs) == 2:
            base_price = ""
            price = ""
            color_fabric = ""
            

        size_elements = text_info.find_elements("xpath",'.//optgroup[@label="사이즈"]/option')
        size = "M"
        is_available = True

        for element in size_elements:
            size = element.text.replace(" [품절]","")
            if "[품절]" in element.text:
                is_available = False
            else:
                is_available = True

            base_dict["baseCat"].append(cat)
            base_dict["baseName"].append(base_name)
            base_dict["baseCode"].append(base_code)
            base_dict["baseUrl"].append(baseURL)
            base_dict["colorName"].append(color_name)
            base_dict["baseGender"].append(0)
            base_dict["price"].append(price)
            base_dict["basePrice"].append(base_price)
            base_dict["colorFabric"].append(color_fabric)
            base_dict["imageUrl"].append(imageURL)
            base_dict["size"].append(size)
            base_dict["isAvailable"].append(is_available)
            
    return base_dict

In [7]:
import pandas as pd

In [8]:
urls = [
        ('https://frizm.co.kr/product/list.html?cate_no=24',
         17), #아우터
        ('https://frizm.co.kr/product/list.html?cate_no=52',
         37), #상의
        ('https://frizm.co.kr/product/list.html?cate_no=26',
         14), #하의
         ]

product_paths = find_product_paths(urls)
print(len(product_paths))

2641


In [9]:
dict1 = crawl_pages(product_paths[:500])
df1 = pd.DataFrame.from_dict(dict1)

print(df1)

     baseCat                          baseName                     baseCode  \
0        top     Paulo drizzler jacket _ beige       Paulo drizzler jacket    
1        top     Paulo drizzler jacket _ beige       Paulo drizzler jacket    
2        top     Paulo drizzler jacket _ beige       Paulo drizzler jacket    
3        top     Paulo drizzler jacket _ black       Paulo drizzler jacket    
4        top     Paulo drizzler jacket _ black       Paulo drizzler jacket    
...      ...                               ...                          ...   
1486     top    Heavy frost down parka _ olive      Heavy frost down parka    
1487     top    Heavy frost down parka _ olive      Heavy frost down parka    
1488     top  Heavy flannel shirt jacket _ red  Heavy flannel shirt jacket    
1489     top  Heavy flannel shirt jacket _ red  Heavy flannel shirt jacket    
1490     top  Heavy flannel shirt jacket _ red  Heavy flannel shirt jacket    

                                                bas

In [10]:
dict2 = crawl_pages(product_paths[500:1000])
df2 = pd.DataFrame.from_dict(dict2)

print(df2)

     baseCat                                           baseName  \
0        top                         M65 fishtail parka _ beige   
1        top                         M65 fishtail parka _ beige   
2        top                         M65 fishtail parka _ beige   
3        top                        Karakoram down parka _ navy   
4        top                        Karakoram down parka _ navy   
...      ...                                                ...   
1185     top  [PENFIELD X FRIZMWORKS] Forest logo sweatshirt...   
1186     top  [PENFIELD X FRIZMWORKS] Forest logo sweatshirt...   
1187     top  [PENFIELD X FRIZMWORKS] Forest logo sweatshirt...   
1188     top  [PENFIELD X FRIZMWORKS] Forest logo sweatshirt...   
1189     top  [PENFIELD X FRIZMWORKS] Forest logo sweatshirt...   

                     baseCode  \
0         M65 fishtail parka    
1         M65 fishtail parka    
2         M65 fishtail parka    
3       Karakoram down parka    
4       Karakoram down parka  

In [11]:
dict3 = crawl_pages(product_paths[1000:1500])
df3 = pd.DataFrame.from_dict(dict3)

print(df3)

     baseCat                               baseName  \
0        top           Double layered hoody _ ivory   
1        top           Double layered hoody _ ivory   
2        top           Double layered hoody _ ivory   
3        top       Double layered hoody _ dark gray   
4        top       Double layered hoody _ dark gray   
...      ...                                    ...   
1490     top  Nylon fatigue shirt jacket _ charcoal   
1491     top  Nylon fatigue shirt jacket _ charcoal   
1492     top     Nylon fatigue shirt jacket _ black   
1493     top     Nylon fatigue shirt jacket _ black   
1494     top     Nylon fatigue shirt jacket _ black   

                         baseCode  \
0           Double layered hoody    
1           Double layered hoody    
2           Double layered hoody    
3           Double layered hoody    
4           Double layered hoody    
...                           ...   
1490  Nylon fatigue shirt jacket    
1491  Nylon fatigue shirt jacket    
1492  

In [12]:
dict4 = crawl_pages(product_paths[1500:2000])
df4 = pd.DataFrame.from_dict(dict4)

print(df4)

    baseCat                            baseName                      baseCode  \
0       top        NYCO Set-up pullover _ olive         NYCO Set-up pullover    
1       top        NYCO Set-up pullover _ olive         NYCO Set-up pullover    
2       top        NYCO Set-up pullover _ olive         NYCO Set-up pullover    
3       top     NYCO Set-up pullover _ charcoal         NYCO Set-up pullover    
4       top     NYCO Set-up pullover _ charcoal         NYCO Set-up pullover    
..      ...                                 ...                           ...   
920     top  FZ Pingpong club sweatshirt _ navy  FZ Pingpong club sweatshirt    
921     top  FZ Pingpong club sweatshirt _ navy  FZ Pingpong club sweatshirt    
922     top  FZ Pingpong club sweatshirt _ gray  FZ Pingpong club sweatshirt    
923     top  FZ Pingpong club sweatshirt _ gray  FZ Pingpong club sweatshirt    
924     top  FZ Pingpong club sweatshirt _ gray  FZ Pingpong club sweatshirt    

                           

In [13]:
dict5 = crawl_pages(product_paths[2000:2500])
df5 = pd.DataFrame.from_dict(dict5)

print(df5)

     baseCat                          baseName                   baseCode  \
0        top          OG Cashmere knit _ ivory          OG Cashmere knit    
1        top          OG Cashmere knit _ ivory          OG Cashmere knit    
2        top          OG Cashmere knit _ ivory          OG Cashmere knit    
3        top        OG Cashmere knit _ oatmeal          OG Cashmere knit    
4        top        OG Cashmere knit _ oatmeal          OG Cashmere knit    
...      ...                               ...                        ...   
1215  bottom          Oxford half pants _ navy         Oxford half pants    
1216  bottom          Oxford half pants _ navy         Oxford half pants    
1217  bottom  Cotton set up half pants _ beige  Cotton set up half pants    
1218  bottom  Cotton set up half pants _ beige  Cotton set up half pants    
1219  bottom  Cotton set up half pants _ beige  Cotton set up half pants    

                                                baseUrl colorName  baseGend

In [14]:
dict6 = crawl_pages(product_paths[2500:])
df6 = pd.DataFrame.from_dict(dict6)

print(df6)

    baseCat                            baseName                     baseCode  \
0    bottom    Cotton set up half pants _ black    Cotton set up half pants    
1    bottom    Cotton set up half pants _ black    Cotton set up half pants    
2    bottom    Cotton set up half pants _ black    Cotton set up half pants    
3    bottom     Round shirt one piece _ mustard       Round shirt one piece    
4    bottom     Round shirt one piece _ mustard       Round shirt one piece    
..      ...                                 ...                          ...   
137  bottom   OG Heavyweight sweat pants _ gray  OG Heavyweight sweat pants    
138  bottom   OG Heavyweight sweat pants _ gray  OG Heavyweight sweat pants    
139  bottom  OG Heavyweight sweat pants _ black  OG Heavyweight sweat pants    
140  bottom  OG Heavyweight sweat pants _ black  OG Heavyweight sweat pants    
141  bottom  OG Heavyweight sweat pants _ black  OG Heavyweight sweat pants    

                                       

In [15]:
df_final = pd.concat([df1, df2, df3, df4, df5, df6])

In [16]:
df_final.to_csv('result.csv',index=False)